## ECI 249 HW \#5

Kenneth Larrieu

### a) Formulate this problem as an optimization problem, minimizing the net expected value cost of managing the floodplain costs and damage. 

The optimal floodplain management options may be found by minimizing the net expected value cost for floodplain management. The next expected value cost is:

$$EVC = \sum_{i=1}^3 c_{P_{i}} X_{P_i} + \sum_{s=1}^{5} p(s) \left( \sum_{j=1}^3 c_{E_{js}} X_{E_{js}} + D_s \right)$$

where 

$X_{P_i}$ is the number of units used for permanent floodplain management option $P_i$

$c_{P_{i}}$ is the annualized unit cost of the option

$p(s)$ is the probability of flow range $s$

$X_{E_{js}}$ is the number of units used for emergency floodplain management option $E_{js}$ at flow range $s$

$c_{E_{js}}$ is the unit cost of that option, and

$D_s$ is the damage incurred by flood s given the applied floodplain management options:

$$D_s = d_{s} - \sum_{i=1}^3 r_{P_i}X_{P_i} + \sum_{j=1}^3 r_{E_{js}} X_{E_{js}} $$

where $d_{s}$ is the damage incurred at flood $s$ in the absence of floodplain management actions

Thus, we want to find the vector $X_{P_i}$ of permanent option quantities and the tensor $X_{E_{js}}$ of temporary option quantities at each discharge range which minimize $EVC$, subject to the imposed bounds on values of their elements.

The restriction of damage to non-negative values can be expressed by the following set of additional constraints on the system:

$$\sum_{i=1}^3 r_{P_i}X_{P_i} + \sum_{j=1}^3 r_{E_{js}} X_{E_{js}}  \leq d_{s}, \quad \forall s$$

where $r$ denotes the reduction in damage per unit $X$.

### b) Minimize the expected value cost of flood damage and control costs by selecting which permanent and short-term flood control measures should be undertaken.  Do this for both current conditions and with the upstream project.

The following information is given regarding the constants for the system:

![](hw5_1.jpg)

![](hw5_2.jpg)

The linear programming problem is then implemented with the Python library `pulp`:

In [5]:
import pulp
import pandas as pd


def dot(l1, l2):
    """Dot product between two lists"""
    return sum([e1 * e2 for e1, e2 in zip(l1, l2)])


class LP:
    def __init__(self, *args):
        self.q_names = ['<5,000 cfs', '5-6,000 cfs', '6-8,000 cfs', '8-10,000 cfs', '10,000+ cfs']
        self.q_probs = [0.8, 0.11, 0.06, 0.02, 0.01]
        if 'upstream' in args:
            self.q_probs = [0.9, 0.05, 0.03, 0.01, 0.01]
        self.damage0 = [0, 2.1e6, 3e6, 4.2e6, 6e6]

        self.perm_names = ['Raise Structures', 'Warning System', 'Sacrificial First Stories']
        self.perm_costs = [10, 1, 40]
        self.perm_lims = [1e6, 200e3, 200e3]
        self.perm_reds = [[0, 100, 70, 60, 10], [0, 2, 3, 4, 7], [0, 100, 60, 50, 20]]
        # self.perm_reds = [[0, 200, 90, 70, 10], [0, 3, 4, 6, 10], [0, 100, 60, 50, 20]] # test case (Jay's paper example)

        self.em_names = ['Evacuate', 'Sandbagging', 'Heightened Levee Monitoring']
        self.em_costs = [200e3, 20e3, 1]
        # self.em_costs = [100e3, 30e3, 1] # test case (Jay's paper example)
        self.em_lims = [1, 2, 20e3]
        self.em_reds = [[0, 200e3, 300e3, 500e3, 1e6], [0, 1e6, 800e3, 0, 0], [0, 2, 1, 0, 0]]
        
        # make set of linear coefficients for optimization
        # index 0-2: permanent options
        self.cs = []
        #print('\nWorthwhile investments:\n')
        for i in range(len(self.perm_names)):
            c = self.perm_costs[i] - dot(self.q_probs, self.perm_reds[i])
            #if c < 0:
                #print(self.perm_names[i])
            self.cs.append(c)
        # index 3-17: em options, listed primarily by q, secondarily by option
        for q in range(len(self.q_names)):
            for j in range(len(self.em_names)):
                c = self.q_probs[q] * (self.em_costs[j] - self.em_reds[j][q])
                #if c < 0:
                    #print(self.em_names[j] + ', ' + self.q_names[q])
                self.cs.append(c)


    def run_LP(self):
        # initialize decision variables
        # list of permanent variables
        perm_vars = [pulp.LpVariable(self.perm_names[i], lowBound=0, upBound=self.perm_lims[i], cat='Integer') for i in range(len(self.perm_names))]
        # list of dicts of emergency variables, dict for each emergency option has flow names as keys
        em_vars = [pulp.LpVariable.dict(self.em_names[j], self.q_names, lowBound=0, upBound=self.em_lims[j], cat='Integer') for j in range(len(self.em_names))]

        self.varlist = perm_vars
        for q in self.q_names:
            for j in range(len(self.em_names)):
                self.varlist.append(em_vars[j][q])

        # initialize model, set to minimize objective fn
        model = pulp.LpProblem('Optimizing Floodplain Management', pulp.LpMinimize)

        # define objective function
        model.objective += pulp.lpSum([self.cs[i] * self.varlist[i] for i in range(len(self.varlist))])
        model.objective += dot(self.q_probs, self.damage0)

        # initialize constraints
        for q in range(1, len(self.q_names)):
            model += pulp.lpSum([self.perm_reds[i][q] * perm_vars[i] for i in range(len(self.perm_names))]) + pulp.lpSum([self.em_reds[j][q] * em_vars[j][self.q_names[q]] for j in range(len(self.em_names))]) <= self.damage0[q]

        model.solve()
        print('\nModel status: %s' % pulp.LpStatus[model.status])
        #print('Model Objective:')
        #print(model.objective)
        #print('\nModel constraints:')
        #print(model.constraints)
        print('\nObjective function value: %.2f' % pulp.value(model.objective))
        print('\nOptimal values:\n')
        for var in model.variables():
            print var
            print var.varValue


        return model



lp = LP()
model = lp.run_LP()



Model status: Optimal

Objective function value: 219300.00

Optimal values:

Evacuate_10,000__cfs
1.0
Evacuate_5_6,000_cfs
0.0
Evacuate_6_8,000_cfs
1.0
Evacuate_8_10,000_cfs
1.0
Evacuate_<5,000_cfs
0.0
Heightened_Levee_Monitoring_10,000__cfs
0.0
Heightened_Levee_Monitoring_5_6,000_cfs
0.0
Heightened_Levee_Monitoring_6_8,000_cfs
0.0
Heightened_Levee_Monitoring_8_10,000_cfs
0.0
Heightened_Levee_Monitoring_<5,000_cfs
0.0
Raise_Structures
1000.0
Sacrificial_First_Stories
0.0
Sandbagging_10,000__cfs
0.0
Sandbagging_5_6,000_cfs
2.0
Sandbagging_6_8,000_cfs
2.0
Sandbagging_8_10,000_cfs
0.0
Sandbagging_<5,000_cfs
0.0
Warning_System
0.0


Optimizing Floodplain Management:
MINIMIZE
-8000.0*Evacuate_10,000__cfs + -6000.0*Evacuate_6_8,000_cfs + -6000.0*Evacuate_8_10,000_cfs + 160000.0*Evacuate_<5,000_cfs + 0.01*Heightened_Levee_Monitoring_10,000__cfs + -0.11*Heightened_Levee_Monitoring_5_6,000_cfs + 0.02*Heightened_Levee_Monitoring_8_10,000_cfs + 0.8*Heightened_Levee_Monitoring_<5,000_cfs + -6.5*Raise_Structures + 24.2*Sacrificial_First_Stories + 200.0*Sandbagging_10,000__cfs + -107800.0*Sandbagging_5_6,000_cfs + -46800.0*Sandbagging_6_8,000_cfs + 400.0*Sandbagging_8_10,000_cfs + 16000.0*Sandbagging_<5,000_cfs + 0.45*Warning_System + 555000.0
SUBJECT TO
_C1: 200000 Evacuate_5_6,000_cfs + 2 Heightened_Levee_Monitoring_5_6,000_cfs
 + 100 Raise_Structures + 100 Sacrificial_First_Stories
 + 1000000 Sandbagging_5_6,000_cfs + 2 Warning_System <= 2100000

_C2: 300000 Evacuate_6_8,000_cfs + Heightened_Levee_Monitoring_6_8,000_cfs
 + 70 Raise_Structures + 60 Sacrificial_First_Stories
 + 800000 Sandbagging_6_8,000_cfs + 3 Warning_Sy

### c) What is the value of the upstream project, in terms of flood damage reduction?

Optimal EVC for downstream - optimal EVC for upstream

### d) For each used and unused decision, what would be the range of unit costs for which these optimized decisions would not change?  Present this as a table.

asdf